# Coursera Peer-graded Assignment
## Segment and Cluster Toronto Neighborhoods

### Step 1: Get postal code data from wikipedia, clean the data and get into a pandas dataframe

#  STEP2:   Scroll to the bottom for step 2 coding

Import libraries to be used 


In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup   # using BeautifulSoup to parse html for data needed for this lab
from urllib.request import urlopen   # library to read url web page

Read the wikipedia web page that contains a table of Toronto posstal codes
Parse the page with BeautifulSoup

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' #URL of the wikipedia page to read
f = urlopen(url)   # open the url
wiki_html = f.read()  # read the data from the page
soup = BeautifulSoup(wiki_html, 'html.parser')  # parse data with BeautifulSoup

Extract the table of postal codes and put into a pandas data frame

In [3]:
table = soup.find_all('table')[0]   #  Extract just the postal code table from the html page
dfs = pd.read_html(str(table))    #pandas gets a list of dataframes for the postal codes
df = dfs[0]   # Get postal codes data from the list into a dataframe

Now clean up the data frame 
drop any row that does not have a Borough name assigned.
replace any Neighbourhood that is not assigned with thet the Borough name

In [4]:

# Get rid of the Borough's that are not assigned

df_x = df[df.Borough != 'Not assigned'].copy()

# replace any not assigned Neighbourhoods with Borough

df_x['Neighbourhood'].loc[df_x['Neighbourhood'] == 'Not assigned'] = df_x['Borough']

df_x.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Finally, group the data by postal code, combine neighbourhoods within postal codes separating them with commas

In [6]:

post_codes = df_x.groupby(['Postcode','Borough'],as_index=False)['Neighbourhood'].apply(','.join).reset_index()  # group postal codes, combine neighborhoods 
post_codes.rename(columns={0:'Neighbourhood'}, inplace=True)

Display the shape of the final data frame

In [7]:
post_codes.shape    #  display the shape of the resulting data frame


(103, 3)

In [9]:
post_codes.head()


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Coursera Peer-graded Assignment
## Segment and Cluster Toronto Neighborhoods

### Step 2: Get latitude and longitude for postal codes

In [10]:
import geocoder

In [11]:
lat_lng_coords = None
latitude = []    # create a list for the latitude
longitude = []  # create a list for the longitude

for index, row in post_codes.iterrows():
#    print(row['Postcode'])
    while(lat_lng_coords == None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(row['Postcode']))
        lat_lng_coords = g.latlng
    latitude.append(lat_lng_coords[0]) 
    longitude.append(lat_lng_coords[1])
    lat_lng_coords = None

post_codes['Latitude'] = latitude
post_codes['Longitude'] = longitude
print('coordinates have been loaded')



coordinates have been loaded


In [12]:
post_codes.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.726245,-79.263670
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.713133,-79.285055
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.696665,-79.260163
